In [1]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket')

In [2]:
from pprint import pprint

In [3]:
from datetime import datetime, timedelta
from decimal import Decimal
from copy import deepcopy
from json import load as json_load
from random import randint, shuffle
from requests import get
from time import sleep, time
from requests import get
from requests_futures.sessions import FuturesSession
from requests.exceptions import ConnectTimeout, ConnectionError, ProxyError, ReadTimeout
from concurrent.futures import as_completed, Future

from crocket.bittrex.bittrex2 import Bittrex, format_bittrex_entry, return_request_input
from crocket.utilities.metrics import calculate_metrics, get_interval_index
from crocket.utilities.time import format_time, convert_bittrex_timestamp_to_datetime, utc_to_local

In [4]:
def process_data(input_data, working_data, market_datetime, interval=60):
    
    entries = []
    
    if not working_data:
        working_data = deepcopy(input_data)
        
    for market in working_data:
        
        working_list = working_data.get(market)
        input_list = input_data.get(market)
        current_datetime = market_datetime.get(market)
            
        last_id = working_list[0].get('Id')
        id_list = [x.get('Id') for x in input_list]
        
        if last_id in id_list:
            overlap_index = id_list.index(last_id)
            working_list = input_list[:overlap_index] + working_list
        else:
            working_list = input_list + working_list
            print('Latest ID in {} working list not found in input data. '
                         'Adding all input data to working list.')
        
        working_data[market] = working_list
        
        latest_datetime = utc_to_local(convert_bittrex_timestamp_to_datetime(working_list[0].get('TimeStamp')))
        
        if (latest_datetime - current_datetime).total_seconds() > interval:
            
            timestamp_list = [utc_to_local(convert_bittrex_timestamp_to_datetime(x.get('TimeStamp')))
                              for x in working_list]
            
            start, stop = get_interval_index(timestamp_list, current_datetime, interval)

            if start == stop:
                while (current_datetime + timedelta(seconds=interval)) < timestamp_list[start - 1]:
                    metrics = calculate_metrics(working_list[start:stop], current_datetime)

                    metrics['price'] = last_price.get(market)

                    fields, values = format_bittrex_entry(metrics)
                    entries.append((market, fields, values))

                    current_datetime = current_datetime + timedelta(seconds=interval)

                market_datetime[market] = current_datetime
            else:
                metrics = calculate_metrics(working_list[start:stop], current_datetime)

                fields, values = format_bittrex_entry(metrics)
                entries.append((market, fields, values))
                
                market_datetime[market] = current_datetime + timedelta(seconds=interval)
                
            working_data[market] = working_list[:start]
            last_price[market] = metrics.get('price')
    
    # db.insert_transaction_query(entries)
    
    return working_data, market_datetime

In [5]:
PROXY_LIST_PATH = '/home/b3arjuden/crocket/proxy_list.txt'
BITTREX_CREDENTIALS_PATH = '/home/b3arjuden/bittrex_credentials.json'

In [6]:
def configure_ip(ip):
    
    return {
        'http': ip,
        'https': ip
    }

def process_response(session, response):
    
    try:
        response.data = response.json()
    except:
        response.data = {
               'success': False,
               'message': 'NO_API_RESPONSE',
               'result': None
            }

In [7]:
# Read files

with open(PROXY_LIST_PATH, 'r') as f:
    proxies = f.read().splitlines()
    
with open(BITTREX_CREDENTIALS_PATH, 'r') as f:
    credentials = json_load(f)

In [8]:
# Create bittrex objects

b1 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             api_version='v1.1')

b2 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             dispatch=return_request_input,
             api_version='v1.1')

In [9]:
# Get list of currencies

response = b1.get_markets()

markets = [x.get('MarketName') for x in response.get('result') 
              if x.get('BaseCurrency') == 'BTC' and x.get('IsActive')]

In [10]:
num_proxies = 200
rind = list(range(num_proxies))

In [11]:
global last_price
last_price = {k:Decimal(0) for k in markets}

In [12]:
# Test asynchronous requests

MAX_API_RETRY = 5

response_dict = {}
working_data = {}

futures = []

current_datetime = datetime.now().astimezone(tz=None)
current_datetime = {k:current_datetime for k in markets}

with FuturesSession(max_workers=10) as session:

    for ii in range(0, 30): # TODO: replace with while loop

        shuffle(rind)
        start = time()

        for index in range(len(markets)):

            market = markets[index]
            request_input = b2.get_market_history(market)

            proxy = configure_ip(proxies[rind[index]])
            url = request_input.get('url')
            headers = {"apisign": request_input.get('apisign')}
            
            response = session.get(url, 
                                   background_callback=process_response,
                                   headers=headers,
                                   timeout=5,
                                   proxies=proxy)
            
            # Add attributes to response
            response.market = market
            response.url = request_input.get('url')
            response.headers = headers
            
            futures.append(response)

        stop = time()
        time1 = stop - start
        print('Async request time: {}s'.format(str(time1)))

        start = time()
        for future in as_completed(futures):
            
            try:
                response_dict[future.market] = future.result().data.get('result')
            except (ProxyError, ConnectTimeout, ConnectionError, ReadTimeout):
                print('Failed API call for {}.'.format(future.market))
                
                api_retry = 0
                
                while True:
                    
                    print('Retrying...')
                    r = randint(0, num_proxies-1)
                    proxy = configure_ip(proxies[r])
                        
                    try:
                        response = session.get(future.url, 
                                               background_callback=process_response,
                                               headers=future.headers,
                                               timeout=3,
                                               proxies=proxy)
                        response_dict[future.market] = response.result().data.get('result')
                        break
                        
                    except (ProxyError, ConnectTimeout, ConnectionError, ReadTimeout):
                        api_retry += 1
                        print("Future status: {}".format(response))
                        print('Retried API call failed.')
                        
                        if api_retry >= MAX_API_RETRY:
                            print('MAX API RETRY LIMIT ({}) REACHED. SKIPPING {}.'.format(str(MAX_API_RETRY), future.market))
                        pass

                print('Retried API call for {} successful.'.format(future.market))
        
        stop = time()
        time2 = stop - start
        print('Futures execution time: {}s'.format(str(time2)))
        
        start = time()
            
        working_data, current_datetime = process_data(response_dict, working_data, current_datetime)
        
        print([len(working_data.get(x)) for x in working_data])
        
        stop = time()
        time3 = stop - start
        print('Processing and uploading data time: {}s'.format(str(time3)))
        
        del futures[:]
        
        if time1 + time2 +time3 < 30:
            sleep(30 - (int(time1) + int(time2) + int(time3)))

Async request time: 0.03355741500854492s
Futures execution time: 16.54775619506836s
[200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 

Async request time: 0.05514335632324219s
Futures execution time: 5.034542798995972s
LOOK HERE!!! start == stop, interval >= 60
[2.224e-05] [3.291e-05] ['BUY']
[0.01656989, 0.01656996, 0.01652858, 0.01652858, 0.01652858, 0.01652858] [0.00999995, 0.00058172, 0.02048081, 0.01026234, 0.03407057, 0.00032227] ['BUY', 'BUY', 'SELL', 'SELL', 'SELL', 'BUY']
[7.219e-05] [0.00100908] ['SELL']
LOOK HERE!!! start == stop, interval >= 60
[0.01822893, 0.01818266, 0.01818266, 0.01815921, 0.01814785, 0.0180834, 0.01805, 0.018, 0.018, 0.018, 0.018, 0.018, 0.01799999, 0.01797783, 0.01797783, 0.01797174, 0.01793661, 0.01793661, 0.01793659, 0.01793657, 0.01793656, 0.01778448, 0.01777379, 0.01777201] [0.00259475, 0.00030152, 0.03280033, 0.00060603, 0.00094108, 0.00118603, 0.0259386, 0.018, 0.10281402, 0.00052812, 0.01975494, 0.24839813, 0.01799999, 0.00121443, 0.06161728, 0.00092917, 0.00269049, 0.03035711, 0.03364927, 0.03587314, 0.09988072, 0.00211435, 0.0010163, 0.00288197] ['BUY', 'BUY', 'BUY', 'BUY', '

[0.144777, 0.144777, 0.144777, 0.144777, 0.14477701, 0.14477701, 0.14477701, 0.14474325, 0.144777, 0.14477701, 0.14477701, 0.14559996, 0.14559996, 0.14546122, 0.144777, 0.144777, 0.1453, 0.144777, 0.144777, 0.14477701, 0.14477701, 0.14477701, 0.14477701, 0.14559998, 0.14559998, 0.14559998, 0.14546122, 0.14546122, 0.14546122, 0.14546122, 0.1456, 0.1456, 0.14474325, 0.14474326, 0.14512661, 0.14549998, 0.1456, 0.1456, 0.14569997, 0.14569997, 0.14569997, 0.1456, 0.1456, 0.14569997, 0.1456, 0.1456, 0.1456, 0.14569998, 0.14569998, 0.1456, 0.1456, 0.14569998, 0.1456, 0.14569998, 0.14569998] [0.0043148, 0.13722299, 0.01158216, 0.58131103, 0.00723885, 0.05112611, 0.00638719, 0.02548015, 0.00862549, 0.06911735, 0.72064873, 0.04367998, 0.13964081, 0.02909224, 0.0434331, 0.0420279, 0.00141027, 0.0434331, 0.0072574, 0.03617569, 0.0434331, 0.0434331, 0.0434331, 0.01686369, 0.06911596, 0.06207542, 0.08402723, 0.00299999, 0.00299508, 0.01732805, 0.00292927, 7.072e-05, 0.04260016, 0.01048419, 0.0029999

KeyboardInterrupt: 

In [ ]:
last_price